<a href="https://colab.research.google.com/github/anitarestrepo16/databases_class_pandas/blob/main/CAPP_121_Pandas_PA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
'''
Helper function for traffic stops assignment
'''
def visualize_rate_series(rate_series, filename='barplot.png'):
    '''
    Purpose: creates a barplot and prints to file
    '''
    graph_df = (rate_series
                .to_frame()
                .reset_index())
    bar_plot = sns.barplot(
        x=graph_df.iloc[:, -2], y=graph_df.iloc[:, -1], color='blue')
    fig = bar_plot.get_figure()
    fig.savefig(filename)


In [ ]:
from pandas.core.arrays import boolean
# Defined constants for column names
ARREST_CITATION = 'arrest_or_citation'
IS_ARRESTED = 'is_arrested'
YEAR_COL = 'stop_year'
MONTH_COL = 'stop_month'
DATE_COL = 'stop_date'
STOP_SEASON = 'stop_season'
STOP_OUTCOME = 'stop_outcome'
SEARCH_TYPE = 'search_type'
SEARCH_CONDUCTED = 'search_conducted'
AGE_CAT = 'age_category'
OFFICER_ID = 'officer_id'
STOP_ID = 'stop_id'
DRIVER_AGE = 'driver_age'
DRIVER_RACE = 'driver_race'
DRIVER_GENDER = 'driver_gender'
VIOLATION = "violation"

SEASONS_MONTHS = {
    "winter": [12, 1, 2],
    "spring": [3, 4, 5],
    "summer": [6, 7, 8],
    "fall": [9, 10, 11]}

NA_DICT = {
    'drugs_related_stop': False,
    'search_basis': "UNKNOWN"
    }

AGE_BINS = [0, 21, 36, 50, 65, 100]
AGE_LABELS = ['juvenile', 'young_adult', 'adult', 'middle_aged', 'senior']

SUCCESS_STOPS = ['Arrest', 'Citation']

CATEGORICAL_COLS = [AGE_CAT, DRIVER_GENDER, DRIVER_RACE,
                    STOP_SEASON, STOP_OUTCOME, VIOLATION]

col_types = {'stop_id': str, 'stop_date': np.datetime64,
             'officer_id': str, 'driver_gender': str,
             'driver_age': np.float64,
             'driver_race': str, 'ethnicity': str,
             'violation': str, 'is_arrested': bool,
             'stop_outcome': str}

In [ ]:
# Task 1a
def read_and_process_allstops(csv_file):
    '''
    Purpose: read in csv and process it according to the assignment
      requirements.

    Inputs:
        csv_file (string): path to the csv file to open

    Returns: (dataframe): a processed dataframe,
      or None if the file does not exist
    '''

    col_types = {'stop_id': str, 'stop_date': str,
             'officer_id': str, 'driver_gender': str,
             'driver_age': int,
             'driver_race': str, 'ethnicity': str,
             'violation': str, 'is_arrested': bool,
             'stop_outcome': str}

    try:
        df = pd.read_csv(csv_file, dtype = col_types)
    except FileNotFoundError:
        return None

    df.loc[:, 'stop_date'] = pd.to_datetime(df.loc[:, 'stop_date'])
    df.loc[:, 'stop_year'] = df.loc[:, 'stop_date'].dt.year
    df.loc[:, 'stop_month'] = df.loc[:, 'stop_date'].dt.month

    SEASONS_MONTHS = {
        "winter": [12, 1, 2],
        "spring": [3, 4, 5],
        "summer": [6, 7, 8],
        "fall": [9, 10, 11]}
    df.loc[:, 'stop_season'] = np.select([df.loc[:, 'stop_month'].isin(SEASONS_MONTHS['winter']),
                                      df.loc[:, 'stop_month'].isin(SEASONS_MONTHS['spring']),
                                      df.loc[:, 'stop_month'].isin(SEASONS_MONTHS['summer']),
                                      df.loc[:, 'stop_month'].isin(SEASONS_MONTHS['fall'])],
                                     ['winter', 'spring', 'summer', 'fall'])

    AGE_BINS = [0, 21, 36, 50, 65, 100]
    AGE_LABELS = ['juvenile', 'young_adult', 'adult', 'middle_aged', 'senior']
    df.loc[:, 'age_category'] = pd.cut(df.loc[:, 'driver_age'], bins = AGE_BINS, labels = AGE_LABELS)

    SUCCESS_STOPS = ['Arrest', 'Citation']
    df.loc[:, 'arrest_or_citation'] = df.loc[:, 'stop_outcome'].isin(SUCCESS_STOPS)

    df.loc[:, 'officer_id'] = df.loc[:, 'officer_id'].fillna('UNKNOWN')

    CATEGORICAL_COLS = ['age_category', 'driver_gender', 'driver_race',
                    'stop_season', 'stop_outcome', 'violation']
    for col_name in CATEGORICAL_COLS:
        df.loc[:, col_name] = df.loc[:, col_name].astype("category")

    return df


In [ ]:
df = read_and_process_allstops('/drive/My Drive/Colab Notebooks/Data/all_stops_basic.csv')
df.head()

<ipython-input-74-05b31f9d798d>:26: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'stop_date'] = pd.to_datetime(df.loc[:, 'stop_date'])
<ipython-input-74-05b31f9d798d>:53: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, col_name] = df.loc[:, col_name].astype("category")
<ipython-input-74-05b31f9d798d>:53: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if 

,stop_id,stop_date,officer_id,driver_gender,driver_age,driver_race,ethnicity,violation,is_arrested,stop_outcome,stop_year,stop_month,stop_season,age_category,arrest_or_citation
0,2168033,2004-05-29,10020,M,53,White,N,Registration/plates,False,Written Warning,2004,5,spring,middle_aged,False
1,4922383,2009-09-04,21417,M,22,Hispanic,H,Other,False,Citation,2009,9,fall,young_adult,True
2,924766,2001-08-13,10231,M,38,White,N,Other,False,Citation,2001,8,summer,adult,True
3,8559541,2014-05-25,11672,F,19,White,N,Other,False,Citation,2014,5,spring,juvenile,True
4,8639335,2014-07-05,21371,F,76,White,N,Other,False,Citation,2014,7,summer,senior,True


In [ ]:
# Task 1b
def read_and_process_searches(csv_file, fill_na_dict=None):
    '''
    Purpose: read in csv and process it according to the assignment
        requirements.

    Inputs:
        csv_file (string): path to the csv file to open
        fill_na_dict (dict): of the form {colname: fill value}

    Returns: (dataframe): a processed dataframe,
      or None if the file does not exist
    '''

    if fill_na_dict is None:
        # Handle fill_na_dict parameter not supplied
        fill_na_dict = NA_DICT

    col_types = {'stop_id': str,
             'search_type': str,
             'contraband_found': bool,
             'search_basis': str,
             'drugs_related_stop': str}
    df = pd.read_csv(csv_file, dtype = col_types).fillna(fill_na_dict)


    return df


In [ ]:
NA_DICT = {
    'drugs_related_stop': False,
    'search_basis': "UNKNOWN"
    }
searches = read_and_process_searches('/drive/My Drive/Colab Notebooks/Data/search_conducted_mini.csv', NA_DICT)
searches.head()

,stop_id,search_type,contraband_found,search_basis,drugs_related_stop
0,4173323,Probable Cause,False,Observation Suspected Contraband,False
1,996719,Incident to Arrest,True,Observation Suspected Contraband,False
2,5428741,Incident to Arrest,False,Other Official Info,False
3,824895,Incident to Arrest,False,Erratic Suspicious Behaviour,False
4,816393,Protective Frisk,False,Erratic Suspicious Behaviour,False


In [ ]:
NA_DICT = {
    'drugs_related_stop': False,
    'search_basis': "UNKNOWN"
    }
all_searches = read_and_process_searches('/drive/My Drive/Colab Notebooks/Data/search_conducted_assignment.csv', NA_DICT)
all_searches.head()

,stop_id,search_conducted,search_type,contraband_found,search_basis,drugs_related_stop
0,0,True,Incident to Arrest,False,UNKNOWN,False
1,5,True,Consent,False,Observation Suspected Contraband,False
2,20,True,Consent,False,Witness Observation,False
3,24,True,Protective Frisk,False,Suspicious Movement,False
4,25,True,Incident to Arrest,False,Observation Suspected Contraband,False


In [ ]:
# Task 2a
def apply_val_filters(df, filter_info):
    '''
    Purpose: apply a value filter to a dataframe

    Inputs:
        df (dataframe)
        filter_info (dict): of the form {'column_name':
            ['value1', 'value2', ...]}

    Returns: (dataframe) filtered dataframe,
      or None if a specified column does not exist
    '''

    for key in filter_info.keys():
        if key not in df.columns:
            return None

    for key, value in filter_info.items():
        df = df.loc[df.loc[:, key].isin(value), :]

    return df


In [ ]:
filt = {'driver_race': ['Black', 'Hispanic'], 'driver_gender': ['M']}
apply_val_filters(df, filt)

,stop_id,stop_date,officer_id,driver_gender,driver_age,driver_race,ethnicity,violation,is_arrested,stop_outcome,stop_year,stop_month,stop_season,age_category,arrest_or_citation
1,4922383,2009-09-04,21417,M,22,Hispanic,H,Other,False,Citation,2009,9,fall,young_adult,True
7,5109631,2009-12-23,11941,M,65,Black,N,Seat belt,False,Citation,2009,12,winter,middle_aged,True


In [ ]:
# Task 2b
def apply_range_filters(df, filter_info):
    '''
    Purpose: apply a range filter to a dataframe

    Inputs:
        df (dataframe)
        filter_info (dict): of the form {'column_name': ['value1', 'value2']}

    Returns: (dataframe) filtered dataframe,
      or None if a specified column does not exist
    '''

    for key in filter_info.keys():
        if key not in df.columns:
            return None

    for key, value in filter_info.items():
        df = df.loc[df.loc[:, key].between(value[0], value[1]), :]

    return df


In [ ]:
range_filter = {'driver_age': (15, 30)}
apply_range_filters(df, range_filter)

,stop_id,stop_date,officer_id,driver_gender,driver_age,driver_race,ethnicity,violation,is_arrested,stop_outcome,stop_year,stop_month,stop_season,age_category,arrest_or_citation
1,4922383,2009-09-04,21417,M,22,Hispanic,H,Other,False,Citation,2009,9,fall,young_adult,True
3,8559541,2014-05-25,11672,F,19,White,N,Other,False,Citation,2014,5,spring,juvenile,True


In [ ]:
# Task 3
def get_summary_statistics(df, group_col_list, summary_col=DRIVER_AGE):
    '''
    Purpose: produce a dataframe of aggregations

    Inputs:
        df (dataframe): the dataframe to get aggregations from
        group_col_list (list of str colnames): a list of columns to group by
        summary_col (str colname): a numeric column to aggregate

    Returns: (dataframe) a dataframe constructed from aggregations
    '''
    grand_mean = df.loc[:, summary_col].mean()
    out = df.groupby(group_col_list).agg({summary_col: ['median', 'mean']})
    out.columns = ['median', 'mean']
    out.loc[:, 'mean_diff'] = out.loc[:, 'mean']-grand_mean

    return out


In [ ]:
all_stops = read_and_process_allstops('/drive/My Drive/Colab Notebooks/Data/all_stops_assignment.csv')
get_summary_statistics(all_stops, ['driver_race', 'driver_gender'])

<ipython-input-74-05b31f9d798d>:26: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'stop_date'] = pd.to_datetime(df.loc[:, 'stop_date'])
<ipython-input-74-05b31f9d798d>:53: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, col_name] = df.loc[:, col_name].astype("category")
<ipython-input-74-05b31f9d798d>:53: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if 

median       mean  mean_diff
driver_race driver_gender                              
Asian       F                31.0  33.793215  -2.038917
            M                32.0  34.537124  -1.295008
Black       F                32.0  34.378343  -1.453789
            M                35.0  36.786237   0.954105
Hispanic    F                30.0  31.496343  -4.335789
            M                29.0  30.970611  -4.861521
Other       F                30.0  32.148493  -3.683639
            M                31.0  33.976765  -1.855367
White       F                32.0  35.017182  -0.814950
            M                35.0  37.108272   1.276140

In [ ]:
# Task 4
def get_rates(df, cat_col, outcome_col):
    '''
    Purpose: returns dataframe of rates given in outcome column

    Inputs:
        df (dataframe)
        cat_col (list) of the column names to group by
        outcome_col (str) column name of outcome column

    Returns: (dataframe) dataframe with the rates for each outcome.
    '''

    if outcome_col not in df.columns:
        return None

    for col in cat_col:
        if col not in df.columns:
            return None

    out = df.groupby(cat_col + [outcome_col]).count().iloc[:, [0]].reset_index(level = -1).pivot(columns = outcome_col)
    out.fillna(0)
    out.loc[:, 'sum'] = out.sum(axis = 1)
    out = np.divide(out, out.loc[:, ['sum']]).drop(columns = 'sum')
    out.columns = out.columns.droplevel()

    return out


In [ ]:
get_rates(all_stops, ['stop_season'], 'arrest_or_citation')

<ipython-input-238-9d5050abd3db>:23: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  out = np.divide(out, out.loc[:, ['sum']]).drop(columns = 'sum')


arrest_or_citation,False,True
stop_season,,
fall,0.196276,0.803724
spring,0.190865,0.809135
summer,0.187372,0.812628
winter,0.212569,0.787431


In [ ]:
get_rates(all_stops, ['age_category', 'driver_gender'], 'is_arrested')

<ipython-input-238-9d5050abd3db>:23: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  out = np.divide(out, out.loc[:, ['sum']]).drop(columns = 'sum')


is_arrested                    False      True
age_category driver_gender                    
juvenile     F              0.994462  0.005538
             M              0.984669  0.015331
young_adult  F              0.990787  0.009213
             M              0.979329  0.020671
adult        F              0.992185  0.007815
             M              0.983407  0.016593
middle_aged  F              0.995749  0.004251
             M              0.989827  0.010173
senior       F              0.998697  0.001303
             M              0.995823  0.004177

In [ ]:
# Task 5
def compute_search_share(
        stops_df, searches_df, cat_col, M_stops=25):
    '''
    Purpose: return a sorted dataframe of cat_cols by share of search
        conducted
    Inputs:
        stops_df (dataframe)
        searches_df (dataframe)
        cat_cols (list) of the column names to group by
        M_stops (int) minimum number of stops to retain

    Returns (dataframe): dataframe of search rates given by cat_col,
      or None if no officers meet M_stops criterion
    '''

    full = pd.merge(stops_df, searches_df, how = 'left', on = 'stop_id')
    full.loc[:, 'search_conducted'] = full.loc[:, 'search_conducted'].fillna(False)
    officers = full.groupby(cat_col).agg({'search_conducted': 'sum'})
    bad_officers = apply_range_filters(officers, {'search_conducted': (M_stops, 10000)}).reset_index()
    bad_officers = bad_officers.loc[bad_officers.loc[:, 'officer_id'] != "UNKNOWN", :]
    if bad_officers.shape[0] == 0:
        return None
    bad_officers_wide = pd.merge(bad_officers.loc[:, cat_col], full, how = 'left', on = cat_col)
    rates = get_rates(bad_officers_wide, cat_col, 'search_conducted')
    rates = rates.sort_values(by = 1, ascending = False)
    return rates


In [ ]:
compute_search_share(all_stops, all_searches, ['officer_id'])

<ipython-input-310-6725937d55d1>:18: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  full.loc[:, 'search_conducted'] = full.loc[:, 'search_conducted'].fillna(False)
<ipython-input-238-9d5050abd3db>:23: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  out = np.divide(out, out.loc[:, ['sum']]).drop(column

search_conducted,False,True
officer_id,,
10797,0.830508,0.169492
11847,0.854839,0.145161
11799,0.871901,0.128099
12039,0.895652,0.104348
10972,0.913043,0.086957
11043,0.923274,0.076726
10008,0.934463,0.065537
11315,0.934555,0.065445
11563,0.944898,0.055102
